В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000

In [4]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

digits = datasets.load_digits() #импортируем датасет

#выделяем признаки и цели
X = digits.data
y = digits.target

#разделяем данные на тренировочные и тестовые (shuffle = False иначе ответ не принимается)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, shuffle = False)

## Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [48]:
# функция подсчета ближайшего к тестовому объекта из обучающей выборки
def equals(X_train, y_train, X_test):
    min_dist = []
    for i in range(1, len(X_train)):
        #считаем расстояние без корня + соответствующее значение цели
        min_dist.append([sum((X_test - X_train[i]) ** 2), y_train[i]])
        min_dist = sorted(min_dist, key=lambda x: x[0]) #сортируем по возрастанию расстояний
        y_pred = min_dist[:1] #записываем в ответ значение целевой переменной с минимальным расстоянием
    return y_pred

pred = []
for i in range(len(X_test)):
    pred.append(equals(X_train, y_train, X_test[i]))


In [50]:
#считаем количество ошибок
k = 0
for i in range(len(pred)):
    if pred[i][0][1] != y_test[i]:
        k += 1
print('Доля ошибок на тестовой выборке ',k/len(pred),'%')

Доля ошибок на тестовой выборке  0.03777777777777778 %


## Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [51]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators = 1000)

rf_model.fit(X_train, y_train)
pred_rf = rf_model.predict(X_test)

k = 0
for i in range(len(pred_rf)):
    if pred_rf[i] != y_test[i]:
        k += 1
print('Доля ошибок на тестовой выборке ',k/len(pred_rf),'%')

Доля ошибок на тестовой выборке  0.06222222222222222 %


In [52]:
#ради интереса посчитал долю ошибок с классификатором из sklearn для сравнения со своими подсчетами
from sklearn.neighbors import KNeighborsClassifier

kn_model = KNeighborsClassifier(n_neighbors = 1)
kn_model.fit(X_train, y_train)
pred_kn = kn_model.predict(X_test)

k = 0
for i in range(len(pred_kn)):
    if pred_kn[i] != y_test[i]:
        k += 1
print('Доля ошибок на тестовой выборке ',k/len(pred_kn),'%')

Доля ошибок на тестовой выборке  0.03777777777777778 %
